# Шпаргалка по SQL

SQL (Structured Query Language) - это язык запросов к базам данных, структурированных особым образом.
Реляционные базы данных - это базы данных с определенными связями между данными (таблицы и связи между таблицами). В таблице (базе данных), обычно представляется тип сущности, каждая строка - это экземпляр объекта, а столбец - это атрибут (признак) объекта. Каждому столбцу присвен тип данных.
Каждая строка имеет свой уникальный ключ. Строки одной таблицы можно связать с другой таблицей. Тогда столбец с ключами другой таблицы будет называться "внешними ключами". Помимо первичных ключей в таблице могут быть естественные ключи, называемые альтернативными.
СУБД (Система управления базами данных) - это языковые и програмные средства, которыми осуществляется доступ к данным.

Учебник по SQL https://sql-academy.org/ru/guide

Онлайн тренажер - https://sql-academy.org/ru/trainer?sort=byId

## Cоздание базы данных на виртуальной машине, организация доступа и примеры запросов.

### Создание виртуальной машины на Yandex Cloud
1. Жмем кнопку "Подключиться" в правом верхнем углу страницы https://cloud.yandex.ru/.
2. Создайте ваше первое облако. Жмем "Создать".
3. В верхнем правом углу жмем "Создать ресурс" и выбираем "виртуальная машина".
4. Вводим имя машины, выбираем Ubuntu. Диск ставим 20 Гб HDD.
5. Параметры вычислительных ресурсов для учебной работы с SQL можно взять следующие:
- платформа "Intel Cascade Lake"
- гарантированная доля vCPU - 5%.
- RAM - 4Гб
- Дополнительно - прерываемая (виртуальная машина будет каждые сутки останавливаться). Так дольше проработает на пробном периоде.
- Логин - нужен будет для доступа, например, log.
- SSH-ключ. Нужно будет ввести для простого доступа без пароля (должен быть заранее создан и установлен на компьютере).
6. Записываем логин и внешний публичный IP адрес. Жмем "Создать BM". Если Яндекс попросит создать платежный аккаунт - создаем.




### Установка СУБД PostgreSQL
1. Заходим в консоль.
2. Попадаем на виртуальную машину. Для этого вводим в консоль следующее: ssh [имя]@[внешний IP адрес], например:
> ssh log@158.160.49.90.
3. Установка описана на странице: https://www.postgresql.org/download/linux/ubuntu/

Копируем (можно за один раз) всё из следующего блока и запускаем в консоли виртуальной машины:

In [ ]:
# Create the file repository configuration:
sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'

# Import the repository signing key:
wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -

# Update the package lists:
sudo apt-get update

# Install the latest version of PostgreSQL.
# If you want a specific version, use 'postgresql-12' or similar instead of 'postgresql':
sudo apt-get -y install postgresql

### Восстановление дампа учебной базы данных
1. Консолью заходим на виртуальную машину:
> ssh log@158.160.49.90.
2. Устанавливаем unzip.
3. Переходим на пользователя postgres:
> sudo -i -u postgres.
4. Скачиваем архив с базой данных:
> wget https://edu.postgrespro.ru/demo-big.zip
5. Распаковываем:
> unzip demo-big.zip
6. Разворачиваем (от 30 минут) дамп базы данных:
> psql -f demo-big-20170815.sql -U postgres
7. Заходим в базу данных:
> psql demo.
8. Пробуем посмотреть список таблиц:
> \d
 Структура базы данных описана на странице https://postgrespro.ru/docs/postgrespro/15/apks02.html

### Использование графического интерфейса DBeaver
Подготовка сервера.
1. Заходим на виртуальную машину
> ssh log@158.160.49.90.
2. Заходим в пользователя postgres:
> sudo -i -u postgres
3. Задаем пароль (появится текстовый запрос пароля),
записываем или запоминаем, он будет нужен при настройке DBeaver:
>\password postgres
4. Редактируем первый конфиг:
> sudo nano /etc/postgresql/15/main/postgresql.conf
Раскомментируем или добавляем новую строку:
listen_addresses = '*'
5. Редактируем второй конфиг:
>sudo nano /etc/postgresql/15/main/pg_hba.conf
Добавляем строку:
host all all 0.0.0.0/0 md5
6. Перезапускаем postgres:
sudo systemctl restart postgresql
7. Проверяем подключение к базе данных `psql -h <REMOTE HOST> -p <REMOTE PORT> -U <DB_USER> <DB_NAME>` (меняем внешний IP 158.160.49.90 на свой):
> psql -h 158.160.49.90 -p 5432 -U postgres.

8. Скачиваем DBeaver:
> sudo snap install dbeaver-ce.
9. Запускаем DBeaver.
10. Создаем новое подключение: Файл -> Новый -> Database Connection -> Далее -> PostgreSQL -> Далее.
11. Настраиваем новое соединение:
Хост: пишем внешний IP адрес виртуальной машины, например 158.160.49.90.
База данных, меняем на demo.
Вводим пароль для пользователя, который создали выше в пункте 3.
Жмем "Готово".
12. Жмем "Подключиться".
Если не получилось, можно попробовать закрыть программу и заново создать подключение. В крайнем случае можно работать из консоли.

### Примеры запросов к базе данных
Её структура на странице https://postgrespro.ru/docs/postgrespro/15/apks02.html.

1. Куда, в какие дни недели и за какое время можно долететь из Волгограда:
SELECT *
FROM   routes
WHERE  departure_city = 'Волгоград';

2. Ближайший рейс, вылетающий из Екатеринбурга в Москву (текущий момент времени демонстрационной базы вызывается функцией `SELECT bookings.now() as now;`):
select * From flights
inner join routes on flights.arrival_airport = routes.arrival_airport 
--select * From routes r 
where departure_city = 'Екатеринбург' and arrival_city = 'Москва' and scheduled_departure > bookings.now()
ORDER BY scheduled_departure
LIMIT    1;

3. 10 бронирований с самой высокой стоимостью:
select book_ref From Bookings
order by total_amount DESC
limit 10

4. В какие города летали пассажиры с самой высокой стоимостью бронирования?
select distinct city From Bookings
inner join tickets on Bookings.book_ref = tickets.book_ref
inner join ticket_flights on tickets.ticket_no = ticket_flights.ticket_no
inner join flights on ticket_flights.flight_id = flights.flight_id
inner join airports on flights.arrival_airport = airports.airport_code
where Bookings.book_ref in (select book_ref From Bookings
order by total_amount DESC
limit 10);